### Creating new layer for customized torch function

In [1]:
from model import RCNN_Modified, create_model, MotionBlur
import torch
from dataset import train_dataset
import matplotlib.pyplot as plt
from config import DEVICE, NUM_CLASSES, NUM_EPOCHS, OUT_DIR
from config import VISUALIZE_TRANSFORMED_IMAGES
from config import SAVE_PLOTS_EPOCH, SAVE_MODEL_EPOCH
from utils import Averager, show_tranformed_image
from tqdm.auto import tqdm
from dataset import train_loader, valid_loader
import torch
import matplotlib.pyplot as plt
import time

Number of training samples: 29
Number of validation samples: 10



/workspaces/breast-tumor-detection/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prog_bar = tqdm(train_loader, total=len(train_loader))


  0%|          | 0/15 [00:00<?, ?it/s]

In [3]:
# initialize the model and move to the computation device
model = RCNN_Modified(pretrained=create_model(NUM_CLASSES))
model = model.to(DEVICE)
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
train_itr = 1
val_itr = 1
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []

In [3]:
motion_blur = MotionBlur(30)

In [4]:
model2 = torch.nn.Sequential(motion_blur, create_model(2))

In [5]:
images, targets = prog_bar.iterable_next()

In [6]:
images = list(image.to(DEVICE, dtype=torch.float) for image in images)
targets = [
    {k: v.to(DEVICE, dtype=torch.int64) for k, v in t.items()} for t in targets
]

In [11]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT
)

In [18]:
images[0].get_device()

0

In [16]:
model2(images[0])

/workspaces/breast-tumor-detection/Model/FasterRCNN/src/model.py:86: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  motion_east = F.conv2d(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument weight in method wrapper_CUDA___conv_depthwise2d)

In [8]:
for data in prog_bar:
        optimizer.zero_grad()
        images, targets = data

        images = list(image.to(DEVICE, dtype=torch.float) for image in images)
        targets = [
            {k: v.to(DEVICE, dtype=torch.int64) for k, v in t.items()} for t in targets
        ]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        train_loss_list.append(loss_value)

        train_loss_hist.send(loss_value)

        losses.backward()
        optimizer.step()

        train_itr += 1

        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")

KeyboardInterrupt: 